In [1]:
import os
import sys
import shutil
import getpass
import codecs

repos_dir = r'/home/akikun/repos'
sys.path.append(repos_dir)
from sak import display as dp
from rs_corpus import jp

password = 'guoyue21'

In [2]:
def tts(text_to_read, wav_path, lang='ja', voice='Hikari', vt_dir=r'/usr/vt'):
    '''
    perform TTS using vtspeech. 
    The output file will be made in {vt_dir}/result/vthenrik/{output_filename}.wav
    
    Args:
        text_to_read (str): text to be read.
        wav_path (path): the output file. 
        lang (str): language. default: ja.
        voice (str): which voice to be used. default: Hikari.
        vt_dir (path): where vtspeech is installed. default: /usr/vt.
        
    '''
    
    # make tts sample.
    wav_filename = os.path.basename(wav_path).replace('.wav', '')
    command = 'sudo -S ' + os.path.join(vt_dir, 'rest', 'vthenrik') \
    + ' --lang' + lang \
    + ' --voice ' + voice \
    + ' --text ' + text_to_read \
    + ' --afile ' + wav_filename

    #password = getpass.getpass()
    #os.system('echo %s | %s' % (password, command))
    !echo {password} | {command}
    
    # copy the wav file to the desired location.
    wav_path_ = os.path.join(vt_dir, 'result', 'vthenrik', os.path.basename(wav_path))
    shutil.move(wav_path_, wav_path)

## make tts samples for iMetricGAN training.

In [3]:
speaker = 'ashley'
tts_dir    = os.path.join(r'/home/akikun/Projects/nele/tts', speaker)
hikari_dir = r'/home/akikun/Corpus/Japanese/02_USS/Female/04_Hikari'

tts_wavs_dir    = os.path.join(tts_dir, 'wavs')
tts_scripts_dir = os.path.join(tts_dir, 'scripts') 

if not os.path.exists(tts_dir):
    os.makedirs(tts_dir)
if not os.path.exists(tts_wavs_dir):
    os.makedirs(tts_wavs_dir)
if not os.path.exists(tts_scripts_dir):
    os.makedirs(tts_scripts_dir)

In [6]:
# split into sentences.
# for i, txt_ in enumerate(txts.split('。'), start=1):
#     i_str = format(i, '03d')
#     txt = txt_.replace('\r', '').replace('\n', '') + '。'
#     if len(txt) > 1:
#         print('{}:{}'.format(i_str, txt))

In [4]:
def loop_per_sentence_jp(txt_path, trs_path):
    ''' make a loop per sentences using str and the corresponding trs '''
    
    # read trans.
    with open(trs_path) as f:
        trss = f.read().split('\n')
        trss.remove('')
    len(trss)

    # read text.
    txts = codecs.open(txt_path,"rb",'shift_jisx0213').read().split('\r\n')

    # show script and transcription per sentence.
    trs_ = '' # transcription.
    txt_ = '' # script.
    for i, trs in enumerate(trss):
#         if i > 10:
#             break

        txt = txts[i]
        trs_ += trs
        txt_ += txt
        if '。' in txt:
            yield txt_, trs_
            trs_ = ''
            txt_ = ''
            

def check_charactors(txt):
    ng_charactors = ['(', ')', '<', '>', '&']
    check = [1 for i in ng_charactors if i in txt]
    is_ok = True
    if len(check) > 0:
        is_ok = False
    return is_ok

In [10]:
## JAPANESE
filelist = [file.replace('.txt', '') for file in os.listdir(os.path.join(hikari_dir, 'text')) if len(file) == 14]
filelist.sort()
for filename in filelist:
    #print('\n>>> processing {} ...'.format(filename))
    #filename = 'hikari-001'
    txt_path = os.path.join(hikari_dir, 'text', filename + '.txt')
    trs_path = os.path.join(hikari_dir, 'trans', filename + '.trs')

    i = 0
    error_list = []
    for txt, trs in loop_per_sentence_jp(txt_path, trs_path):
        i += 1
        i_str = format(i, '03d')
        basename = filename.replace('hikari', speaker) + '_' + i_str
        print('=== {} ==='.format(basename))
        print('{}'.format(txt))

        if not check_charactors(txt):
            print('=== {} ==='.format(basename))
            print('{}'.format(txt))

        if check_charactors(txt):
            tts(txt, os.path.join(tts_dir, 'wavs', basename + '.wav'), voice='Risa')
            with open(os.path.join(tts_dir, 'scripts', basename + '.txt'), 'w') as f:
                f.write(txt)    
            with open(os.path.join(tts_dir, 'scripts', basename + '.trs'), 'w') as f:
                f.write(trs)
        else:
            error_list.append(basename)
            print('+++++ cannot generate audio +++++')

=== risa-001_001 ===
米側は「極めて高性能なレーダーを持つ護衛艦」との表現でイージス艦派遣を要請、政府も派遣する方針。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-001_001.wav
=== risa-001_002 ===
覇者というのはどういう言葉かというと、王者よりワンランク下の称号です。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-001_002.wav
=== risa-001_003 ===
リンクが無料開放され、九千八百人余が訪れた三日のエムウェーブ。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-001_003.wav
=== risa-001_004 ===
個人の努力や国家のプレッシャーやエリート教育がトップ・アスリートを育てる時代は、もはや終わった。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-001_004.wav
=== risa-001_005 ===
「モアイ」は、イースター島の石像モアイに由来し「未来を生きる」という意味。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-001_005.wav
=== risa-001_006 ===
行政や漁業者、住民がより一層連携を強め地域一体となったクリーン運動を続けていきたい。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file ri

=== risa-001_049 ===
三月十六日の通り魔事件後の四月初めごろ、少年がリポート用紙に手書きし友人にワープロ打ちさせたらしい。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-001_049.wav
=== risa-001_050 ===
先月の開場記念イベントで、日本リーグ一部の女子ソフトボールチームを招いたのもワークショップの成果だった。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-001_050.wav
=== risa-001_051 ===
そこでは、ブルドーザーやショベルカーが音をたてて国有林を切り開き、酪農や養豚など畜産農場の造成工事が急ピッチで進んでいた。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-001_051.wav
=== risa-001_052 ===
ところが、ここ数年はキーを抜いてドアをロックした「キーなし」車の盗難が増加。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-001_052.wav
=== risa-001_053 ===
パキスタンのアフガン政策は、カシミール地方で領有権を争うインドをにらんだものだ。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-001_053.wav
=== risa-001_054 ===
結局、「イングリッシュ・ペーシェント」は最優秀監督、音楽、美術など十二部門、また「シャイン」も七部門にノミネートされた。
[sudo] password for akikun: Norm

=== risa-001_096 ===
生まれ年も同じベビーブーマーだが、両氏の世界観の違いは大きい。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-001_096.wav
=== risa-001_097 ===
そこのところにポイントを絞られたというふうに考えてよろしいんでしょうか。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-001_097.wav
=== risa-001_098 ===
十一回目の今年度のテーマは「好きなひと」、「読む」、「『百聞』と『一見』」。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-001_098.wav
=== risa-001_099 ===
恒久減税を求める声にも「恒久的減税」で逃げ、住宅ローン利子控除も見送られた。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-001_099.wav
=== risa-001_100 ===
ヘリテージ財団のイェーツ研究員は、「逆にゴア陣営への有権者の反発を招く可能性がある」との見方だ。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-001_100.wav
=== risa-001_101 ===
冷戦直後のサミットの場で東欧やロシアを支援する「ヨーロッパ復興開発銀行」が生まれた。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-001_1

=== risa-002_040 ===
「操業権」になれば、原油はクウェート政府が所有し、生産委託を受けたアラ石が歩合収入を得る形になる模様だ。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-002_040.wav
=== risa-002_041 ===
オープニングセレモニーでは、名瀬市の平田隆義市長が、「交流展を通じて沖縄、奄美の文化が大きくなってゆくよう願っています」。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-002_041.wav
=== risa-002_042 ===
スプーンでひとさじすくった瞬間、鼻をくすぐる風味は紛れもなく赤ピーマンそのもの。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-002_042.wav
=== risa-002_043 ===
「テロ組織をそれぞれ打ち負かすまで戦いをやめない」と戦争続行を宣言。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-002_043.wav
=== risa-002_044 ===
「ウォーム」と呼ばれる種類で、インターネットのウェブサイトで増殖し続ける。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-002_044.wav
=== risa-002_045 ===
下草、雑草の駆除目的の注文が多いが、情操教育用にメスの子ヤギを頼んだ幼稚園もあった。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Suc

[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-002_087.wav
=== risa-002_088 ===
研修会では県政策マーケティング委員会委員長の中橋勇一氏が、「行政評価の重要性」と題して講演、評価を数値で示すことの重要性などを強調した。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-002_088.wav
=== risa-002_089 ===
強者優遇、それでは、消費者にとってビッグバンは歓迎できるものになるのか。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-002_089.wav
=== risa-002_090 ===
泉質はナトリウム炭酸水素塩・塩化物温泉で、神経痛や筋肉痛などに効能がある。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-002_090.wav
=== risa-002_091 ===
当初の内容は会社概要、事業案内、プレゼント付きアンケートなど。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-002_091.wav
=== risa-002_092 ===
輸出の狙いは「県産米の販路拡大と知名度アップ、それに生産者の意欲を高める面も大きい」と矢萩次長。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-002_092.wav
=== risa-002_093 ===
いわゆる「和田

...

=== risa-098_023 ===
気まぐれなマーケットはいつ、これを材料に、国債を売り浴びせてくるか分からない。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-098_023.wav
=== risa-098_024 ===
一方のスレイマン農業相は、発生のパターンが、農林畜産国を機能停止に追い込むための手段とした細菌戦争に酷似していると語ったという。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-098_024.wav
=== risa-098_025 ===
レーガン政権の思い切った税制改革と起業優遇規制解除が生きたのである。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-098_025.wav
=== risa-098_026 ===
若いデザイナーたちの間にもそういう要求は高まっていると思うと語った。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-098_026.wav
=== risa-098_027 ===
これに関連してフライシャー米大統領報道官は同日、「米軍は世界各地の平和維持に関与し過ぎてはならない。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-098_027.wav
=== risa-098_028 ===
大阪では昆布の加工業が発達し、とろろ昆布のみそ、塩昆布のお茶漬けなどが庶民の食卓にはポピュラーとなった。
[sudo] password for akikun: Normal file mode. Using encoding

[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-098_070.wav
=== risa-098_071 ===
ですから、今はやりのカフェはフランスでいうところのサロンに近いと思います。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-098_071.wav
=== risa-098_072 ===
集票力のある組織や大物タレントを多く抱えている政党には有利な制度と言える。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-098_072.wav
=== risa-098_073 ===
同時に、日米や中国、ロシア、欧州諸国などが、包容政策を全面的に支持してくれたおかげでもあると感謝した。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-098_073.wav
=== risa-098_074 ===
アレルギーは、食事だけでなく、他の環境要因、便利さと裏腹に激増してきた。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-098_074.wav
=== risa-098_075 ===
ルビー、サファイヤ、トパーズといった宝石の原石が、日本の各地で採取できることはあまり知られていない。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-098_075.wav
=== risa-098_076 ===
品のない質問はひんしゅくを買うが

[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-099_006.wav
=== risa-099_007 ===
インターネット関連が多い一方で「製造業を中心とする業界情報を。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-099_007.wav
=== risa-099_008 ===
知人のネパール人ら四人を同居させ、家賃五万円を折半させていた。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-099_008.wav
=== risa-099_009 ===
店舗には携帯電話販売用の専用コーナーを設け、無料充電サービスも始めた。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-099_009.wav
=== risa-099_010 ===
いつも「ガボッ、ガボッ」とやる出す場所を、何度も行ったり来たりする。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-099_010.wav
=== risa-099_011 ===
派手なネオンと違って、やみ夜に鈍い灯ろうの明かりが懐かしい風情を漂わす。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-099_011.wav
=== risa-099_012 ===
父ヨセフ、母マリア、ですめば簡単なんですが、これが意外とややこしい。
[sudo] password for akiku

Successfully created file risa-099_054.wav
=== risa-099_055 ===
ソニーや松下電器産業など他の電機メーカーにも、同じような表彰製度がある。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-099_055.wav
=== risa-099_056 ===
四輪車販売は主に日、米、欧を拠点とし、二輪車は新市場・中国を含めたアジアで成長を目指す。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-099_056.wav
=== risa-099_057 ===
今朝、リハビリに向かう途中、同級生の実家の造花問屋を覗くと、もうクリスマス用品一色である。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-099_057.wav
=== risa-099_058 ===
右ウイングで出場した小野は、ご存じのとおり中盤の選手でドリブルよりもパスが持ち味。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-099_058.wav
=== risa-099_059 ===
総資産百三十兆円を超す世界一の規模の「みずほ」をはじめ、四大グループはいずれも世界有数のメガバンクだ。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-099_059.wav
=== risa-099_060 ===
銀行、郵便局、映画館、ダンスクラブ、ビリヤード、ゲームセンターなども用意され、各国選手が友好を深め合う場となる。
[sudo] password f

=== risa-099_102 ===
車中で寝泊まりし、レンタカーの使用延長の電話を入れながら、一般道を太平洋沿いに北上した。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-099_102.wav
=== risa-099_103 ===
田園空間のシンボルでもある新井田川では、ちょうど河川改修の真っ最中。
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file risa-099_103.wav


In [5]:
## ENGLISH
# get script from cmu script.
cmu_script_txt = r'/home/akikun/Corpus/cmu/cmu_us_rms_arctic/etc/txt.done.data-all'
with open(cmu_script_txt) as f:
    cmu_scripts = f.read().split('\n')
    cmu_scripts.remove('')
    
for line in cmu_scripts:
    sentence_id = line.split(' ')[1].replace('arctic_', '')
    sentence = line.split('"')[1]
    print('{}: {}'.format(sentence_id, sentence))
    
    tts(
        sentence.replace(' ', '\ ').replace('\'', '\\\'').replace(';', '\;'),
        os.path.join(tts_wavs_dir, sentence_id + '.wav'),
        voice=speaker,
        lang='en_us'
         )
    with open(os.path.join(tts_scripts_dir, sentence_id + '.txt'), 'w') as f:
        f.write(sentence)

a0001: Author of the danger trail, Philip Steels, etc.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0001.wav
a0002: Not at this particular case, Tom, apologized Whittemore.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0002.wav
a0003: For the twentieth time that evening the two men shook hands.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0003.wav
a0004: Lord, but I'm glad to see you again, Phil.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0004.wav
a0005: Will we ever forget it.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0005.wav
a0006: God bless 'em, I hope I'll go on seeing them forever.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0006.wav
a0007: And you always want to see it in the superlative degree.
[sudo] 

a0053: Suddenly his fingers closed tightly over the handkerchief.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0053.wav
a0054: There was nothing on the rock.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0054.wav
a0055: Philip stood undecided, his ears strained to catch the slightest sound.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0055.wav
a0056: Pearce's little eyes were fixed on him shrewdly.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0056.wav
a0057: I have no idea, replied Philip.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0057.wav
a0058: I came for information more out of curiosity than anything else.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0058.wav
a0059: His immaculate appearance was gone.
[sudo] 

[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0106.wav
a0107: If you only could know how I thank you.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0107.wav
a0108: He waded into the edge of the water and began scrubbing himself.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0108.wav
a0109: Do you know that you are shaking my confidence in you.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0109.wav
a0110: Much, replied Jeanne, as tersely.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0110.wav
a0111: Instead, he joined her; and they ate like two hungry children.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0111.wav
a0112: He was wounded in the arm.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successful

[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0158.wav
a0159: They look as though he had been drumming a piano all his life.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0159.wav
a0160: You want to go over and see his gang throw dirt.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0160.wav
a0161: Take away their foreman and they wouldn't be worth their grub.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0161.wav
a0162: That's the sub-foreman, explained Thorpe.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0162.wav
a0163: Philip made no effort to follow.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0163.wav
a0164: He came first a year ago, and revealed himself to Jeanne.
[sudo] password for akikun: Normal file mode. Usin

a0212: Fresh cases, still able to walk, they clustered about the spokesman.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0212.wav
a0213: Between him and the beach was the cane-grass fence of the compound.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0213.wav
a0214: Besides, he was paid one case of tobacco per head.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0214.wav
a0215: They die out of spite.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0215.wav
a0216: The other felt a sudden wave of irritation rush through him.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0216.wav
a0217: Oppressive as the heat had been, it was now even more oppressive.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file a0217.wav
a0218: The ringing o

...

b0304: His previous wives had never lived long enough to bear him children.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0304.wav
b0305: It was our river emerging like ourselves from the great swamp.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0305.wav
b0306: Cameron looked at his hands with their long, sinewy fingers.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0306.wav
b0307: We got few vegetables and fruits, and became fish eaters.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0307.wav
b0308: We never made another migration.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0308.wav
b0309: Nor was Elam Harnish an exception.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0309.wav
b0310: A little treatment, massage, wit

b0358: The temperature dropped to fifty below zero and remained there the whole trip.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0358.wav
b0359: And as never before, he was ready to obey.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0359.wav
b0360: This state of mind comes of an undue prominence of the ego.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0360.wav
b0361: There are the canals of China, and the Yang-tse River.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0361.wav
b0362: We threaten to be of the one mind before the voyage is completed.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0362.wav
b0363: She was built primarily to sail.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0363.wav
b0364: In the Bohemian

Successfully created file b0409.wav
b0410: Have you ever earned a dollar by your own labour.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0410.wav
b0411: He gave no reason, but his motive was obvious enough.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0411.wav
b0412: Ah, it was sweet in my ears.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0412.wav
b0413: He may desire to escape pain, or to enjoy pleasure.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0413.wav
b0414: It was impossible to hoist sail and claw off that shore.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0414.wav
b0415: There was nothing for us but the wide raw ocean.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0415.wav
b0416: I arose obediently and w

Successfully created file b0461.wav
b0462: One guess will do, Ernest retorted.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0462.wav
b0463: Take my advice and accept the vacation.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0463.wav
b0464: I could not agree with Ernest.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0464.wav
b0465: But such divergence of opinion would constitute no menace to society.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0465.wav
b0466: It is dog eat dog, and you ate them up.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0466.wav
b0467: Let us run them for ourselves.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0467.wav
b0468: It was introduced by Representative Dick of Ohio.
[sudo] password f

[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0513.wav
b0514: Tomorrow or next day it might he gone.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0514.wav
b0515: But already he had composed himself.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0515.wav
b0516: Zilla relaxed her sour mouth long enough to sigh her satisfaction.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0516.wav
b0517: Eggshell is not good to eat.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0517.wav
b0518: But there was also talk of witchcraft in the village.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0518.wav
b0519: Yea, I will tell thee.
[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file b0519.wav
b0520: Ha

In [17]:
sentence.replace(' ', '\ ').replace('\'', '\\\'')

'The\\ gray\\ eyes\\ faltered;\\ the\\ flush\\ deepened.'

In [19]:
tts(
    sentence.replace(' ', '\ ').replace('\'', '\\\''),
    os.path.join(tts_wavs_dir, sentence_id + '.wav'),
    voice=speaker,
    lang='en_uk'
     )

[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file vthenrik.wav
/bin/bash:  the flush deepened.: command not found


FileNotFoundError: [Errno 2] No such file or directory: '/usr/vt/result/vthenrik/a0076.wav'

In [27]:
#cmu_scripts.remove('')
cmu_scripts[-3:]

['( arctic_b0537 "But she had become an automaton." )',
 '( arctic_b0538 "At the best, they were necessary accessories." )',
 '( arctic_b0539 "You were making them talk shop, Ruth charged him." )']

## make demo samples for JR.

In [6]:
wav_dir  = r'/home/akikun/Projects/nele/tts/comparison'
wav_path = os.path.join(wav_dir, 'ashley.wav')
script = 'Ladies and gentlemen, welcome to the Shinkansen. This is Kodama super express bound for Shin-Osaka.'
tts(
     #'まもなく３番線に、のぞみ3号博多行きが参ります。危ないので、白線の内側までお下がりください。', 
    #'JR東海をご利用くださいまして、ありがとうございます。',
    #'この電車は、のぞみ3号博多行きです。グリーン車は4号車と5号車です。',
    #'今日も、新幹線をご利用くださいまして、ありがとうございます。この電車は、こだま号、新大阪行きです。',
    script.replace(' ', '\ '),
    wav_path,
    voice='ashley',
    lang='en_us'
     )
dp.disp_wav(wav_path)

[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file ashley.wav
